# Result Visualizer

Different visualizations of anomaly detection result, including forecasts, reconstructions, anomaly scores, predicted and actual anomalies

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import os

import pickle
from sklearn.metrics import f1_score, precision_score, recall_score
import plotly as py
from plotly.offline import download_plotlyjs, init_notebook_mode
import plotly.graph_objs as go
import cufflinks as cf
import glob
import sys
import json
import more_itertools as mit
from tqdm.notebook import tqdm
cf.go_offline()
init_notebook_mode

sys.path.insert(0, '.')
from plotting import Plotter

In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

### Provide what output to visualize
In case of multiple models trained for the same dataset, they will be saved in different folders named using the datetime of training.
Use 'model_id' to specify which to load, '-1' for latest, '-2' for second latest, etc, or you can specify the exact datetime. 

In [ ]:
res_path = './output/MSL'
#res_path = './output/SMAP'
#res_path = './output/SMD/1-1'

In [ ]:
plotter = Plotter(res_path, model_id='-1')
plotter.result_summary()

## Interactive plot 
- Forecasted value, reconstructed value and actual value for one channel (feature) are plotted in the first plot. You can also specify start and end to limit the x-axis.
- Time steps with actual (true) anomalies are visualized by a light red rectangel, while predicted anomalies by a light blue rectangel. If anomalies are predicted correctly the color of the rectangle will therefore be purple. 
- In the second plot the anomaly score (error) is visualized, together with the threshold used to predict anomalies. To toggle between the error of the channel and the total error for all channels, use ```show_tot_err```. 

In [ ]:
plotter.plot_feature(
    feature=0, 
    plot_train=True, 
    plot_errors=True, 
    plot_feature_anom=True,
    start=None, 
    end=None
)

In [ ]:
plotter.plot_global_predictions(type="test")

In [ ]:
plotter.plotly_global_predictions(type="test")

In [ ]:
# plotter.plot_all_features(type='test')